In [1]:
import numpy as np

# Problem 1: Power iteration method for eigenvalue calculation (3 pts)

### <div align="right"> &copy; Volodymyr Kuchynskyi & Rostyslav Hryniv, 2023 </div>

## Completed by:
*   Nazar Andrushko
*   Roman Kovalchuk

---

#### In this part of the homework, you will implement the **power method**, a simple iterative algorithm for numerical calculation of the dominant eigenvalue and the corresponding eigenvector of a square matrix $A$, test its limitations, and verify necessary conditions on $A$. You will also use the **inverse power method**, a modification of the regular **power method**, that finds the remaining non-dominant eigenvalues and eigenvectors of $A$. For simplicity, we will be working with real matrices only  

---

##1. Power iteration method (1 pt)##

###1.1 Explanation of the method
#### Assume that a $k\times k$ matrix $A$ is diagonalizable and that $\lambda_1, \lambda_2, \dots, \lambda_k$ are its eigenvalues listed according to multiplicities. We say that $\lambda_1$ is a <font color="red">dominant eigenvalue</font> of $A$ if the eigenvalues can be ordered so that $|\lambda_1|> |\lambda_2| \ge \dots \ge |\lambda_k|$. In particular, the dominant eigenvalue must be <font color = "blue"> simple</font>; we denote by $\mathbf{v}_1$ a corresponding normalized eigenvector. Analysis of the large-$n$ asymptotics of $A^n\mathbf{x}_0$ for a generic vector $\mathbf{x}_0\in \mathbb{R}^k$ suggests a simple <font color="red">power iteration</font> method to find the eigenspace $\mbox{ls}\{\mathbf{v}_1\}$ and the dominant eigenvalue $\lambda_1$.  

#### Denote by $\mathbf{v}_j$ an eigenvector for the eigenvalue $\lambda_j$, $j=2,\dots, k$. Assume also that the starting vector $\mathbf{x}_0$ has a nonzero component in the direction of $\mathbf{v}_1$, i.e., that in the representation $$
	\mathbf{x}_0 = c_1 \mathbf{v}_1 + c_2 \mathbf{v}_2 + \cdots + c_k\mathbf{v}_k
$$ we have $c_1 \ne 0$. Then $$
	A^n \mathbf{x}_0 = \lambda_1^n \Bigl[c_1\mathbf{v}_1 + c_2 \Bigr(\frac{\lambda_2}{\lambda_1}\Bigr)^n \mathbf{v}_2 + \cdots + c_k \Bigr(\frac{\lambda_k}{\lambda_1}\Bigr)^n\mathbf{v}_k\Bigr] =  \lambda_1^n \bigl[c_1 \mathbf{v}_1 + o(1)\bigr]
$$ as $n \to \infty$. The latter relation still does not allow to identify $\lambda_1$ and $\mathbf{v}_1$ because the term $\lambda_1^n$ blows up when $|\lambda_1|>1$, decays to zero when $|\lambda_1|<1$, and rotates over the unit circle $|z|=1$ when $|\lambda_1|  = 1$ is different from $1$. To compensate that behavior, one iterates over the normalized vectors $\mathbf{x}_n$ defined via $$
	\mathbf{x}_{n} := \frac{A\mathbf{x}_{n-1}}{\|A\mathbf{x}_{n-1}\|} = \frac{A^n\mathbf{x}_0}{\|A^n\mathbf{x}_0\|} = \frac{c_1}{|c_1|} \Bigl(\frac{\lambda_1}{|\lambda_1|}\Bigr)^n\mathbf{v}_1 + o(1), \tag{1}
$$ whose distance to the eigenspace $\mbox{ls}\{\mathbf{v}_1\}$ decays exponentially. The eigenvalue $\lambda_1$ can be approximated by $$
	\lambda_1 \approx \frac{\mathbf{x}_n^\top A \mathbf{x}_n}{\mathbf{x}_n^\top \mathbf{x}_n} = \mathbf{x}_n^\top A \mathbf{x}_n. \tag{2}
$$ Formulae (1) and (2) lay in the basis of the method.


### **1.2 (0.5 pt)**  Implement the power iteration method

> **Note:** Although the use of any function from numpy is allowed, in this part of your homework, methods such as ``power_method`` and ``inverse_power_method`` must be implemented explicitly, without relying on functions that could use them implicitly in their implementations, such as ``np.linalg.eig``. However, you can use the latter in the rest of your homework to e.g. verify the correctness of implemented functions

> **Hint:** The stopping criterion for the iteration (1), i.e., $\mathbf{x}_{n} := {A\mathbf{x}_{n-1}}/{\|A\mathbf{x}_{n-1}\|}$ should be expressed in terms of stabilizing the corresponding eigenvalue $\lambda_1 \approx \mathbf{x}_n^\top \mathbf{x}_{n-1}$. The reason is that if $\lambda_1$ is not positive, then, on each iteration, the vector $\mathbf{x}_n$ gets multiplied by the number $\lambda_1/|\lambda_1|$, which prevents $\mathbf{x}_n$ from converging (cf. (1))

In [2]:
def power_method(A, start_vector=None, tol = 1e-5, max_iter=5e+2):
    """
    Return the dominant eigenvalue and its corresponding eigenvector
    using power method.

    Args:
        A - matrix for which to compute the eigenpair
        start_vector(optional) - vector used for initialization
                                 on the first step of power iteration algorithm.
                                 Defaults to None. In such case, the initial vector
                                 will be randomized.
        tol (optional) - stopping criterion: stop iterations when lambda_1 update
                                 gets smaller than tol
        max_iter(optional) - maximum number of iterations for the power method.
                                 Perform sanity check if the returned values are
                                 close to eigenvalue and eigenvector of A
    Returns:
        (eigval, eigvec) - a pair of the dominant eigenvalue and its eigenvector.
    """

    #YOUR CODE STARTS HERE
    if not start_vector:
        # Choose it randomly to reduce a chance of having it orthogonal to eigenvector
        start_vector = np.random.rand(A.shape[1])

    l_approximated = np.inf

    for _ in range(int(max_iter)):
        b_k1 = np.dot(A, start_vector)
        b_k1_norm = np.linalg.norm(b_k1)
        b_k = b_k1 / b_k1_norm

        # Use Rayleigh quotient to get associated eigenvalue from found eigenvector
        l = np.dot(np.dot(A, b_k), b_k) / np.dot(b_k, b_k)

        if np.abs(l - l_approximated) < tol:
            break

        start_vector = b_k
        l_approximated = l

    return l, b_k

    #YOUR CODE ENDS HERE


In [3]:
A_1 = np.array([[0, 1], [-2, -3]])
#You should invoke similar calls in the next tasks by yourself
A_1_eival, A_1_eivec = power_method(A_1)
print(f"eigenvalue: {A_1_eival}, eigenvector: {A_1_eivec}")

eigenvalue: -2.00000841104366, eigenvector: [ 0.44721109 -0.89442844]


#### Test your implementation of power method:

In [4]:
def test_power_method(A, eigenvalue, eigenvector):

    eigenvals_ref, eigenvecs_ref = np.linalg.eig(A)
    eigenvecs_ref = eigenvecs_ref.T

    eig_imax = np.argmax(np.abs(eigenvals_ref))
    #compare eigenvalues
    assert np.allclose(eigenvalue, eigenvals_ref[eig_imax]),\
                       f"Incorrect eigenvalue found: {eigenvalue} differs from {eigenvals_ref[eig_imax]}"
    #compare eigenvectors w.r.t scalar multiple (normalize)
    assert np.allclose(eigenvector / np.linalg.norm(eigenvector),
                       eigenvecs_ref[eig_imax]) or np.allclose(-(eigenvector / np.linalg.norm(eigenvector)),
                       eigenvecs_ref[eig_imax]),\
                       f"Incorrect eigenvector found: {eigenvector} is not a constant multiple of {eigenvecs_ref[eig_imax]}"

    print("test_power_method passed successfully")

In [5]:
A_test_3x3 = (100*np.random.rand(3,3))
A_test_5x5 = (100*np.random.rand(5,5))
A_test_10x10 = (100*np.random.rand(10,10))

test_power_method(A_test_3x3, *power_method(A_test_3x3))
test_power_method(A_test_5x5, *power_method(A_test_5x5))
test_power_method(A_test_10x10, *power_method(A_test_10x10))

test_power_method passed successfully
test_power_method passed successfully
test_power_method passed successfully


---
### **1.3. (0.5 pt)** Reasons to fail
#### Formulate necessary conditions for power method to work and the reasons why it can fail (the more, the better, but at least two). Then, for each reason, provide an example of your own $3 \times 3$ matrix $M$ when the method fails and test it by your code.
>_Hint_: Recall that for real matrices, the eigenvalues come in complex conjugate pairs. Recall also that not all matrices are diagonalizable; do you see what obstacle that can create?

---
#### **Your explanations come here**
There is a two main reasons why the power method might not work, or produce poor results:
1) Dominant eigenvalue is very close to other eigenvalues
2) The initial assumption of a vector is orthogonal to the A matrix's true eigenvector
given matrix:
[[3, 5, 0],
 [0, -4, 0],
 [0, -4, -4]]
if we choose initial eigenvector guess as [1, 0, 0] (Real eigenvector for eigenvalue = -4 with vector (0,0,1));
we can see the method produces smaller eigenvalue (3), whilst the dominating one is -4, with eigenvector for lambda equals to 3
3) The matrices eigenvalues has the same spectral radius:
For instance if we choose matrix:
[[3, 5, 0],
 [5, -3, 0],
 [0, -4, 3]]
We would see that true eigenvalues are sqrt(34) and -sqrt(34) and 3
Whilst power method produces various values pointing to one of three eigenvectors, but overall, result is far from truth
3) Low number of iterations or high tolerance rate
Depending on number of iterations or tolerance rate set, the method may not produce accurate results, as can be shown from
example at wikipedia page where first few iterations may point out to the direction of smallest eigenvector to say.
---

In [23]:
#YOUR CODE STARTS HERE
B_bad_initial_guess = np.array(
[[3, 5, 0],
 [0, -4, 0],
 [0, -4, -4]])
display(test_power_method(B_bad_initial_guess, *power_method(B_bad_initial_guess, start_vector=[1, 0, 0])))
#YOUR CODE ENDS HERE

AssertionError: Incorrect eigenvalue found: 3.0 differs from -4.0

In [26]:
B_spectral_radius = np.array(
[[3, 5, 0],
 [0, -3, 0],
 [0, -4, 3]])
display(test_power_method(B_spectral_radius, *power_method(B_spectral_radius)))

AssertionError: Incorrect eigenvalue found: 1.4837451643372273 differs from 3.0

## 2. Symmetric matrices (1 pt)##

###2.1. Recap on symmetric matrices
#### Consider a special case of finding eigenvalues and eigenvectors for a **symmetric** matrix $A$, i.e. a matrix satisfying $A^\top = A$. Recall that such a matrix  
- is **orthogonally diagonalizable**, i.e., there is an **orthonormal basis** $\mathbf{v}_1, \dots,\mathbf{v}_k$ of $\mathbb{R}^k$ consisting of **eigenvectors** of $A$;
- has only real eigenvalues $\lambda_1, \lambda_2, \dots, \lambda_k$;
- can be written as $$A = \lambda_1 \mathbf{v}_1\mathbf{v}_1^\top + \lambda_2 \mathbf{v}_2\mathbf{v}_2^\top + \dots + \lambda_k \mathbf{v}_k\mathbf{v}_k^\top$$
by the **spectral theorem**  

####Assume that $|\lambda_1|\ge |\lambda_2| \ge \dots  |\lambda_k|$ and that $|\lambda_j| = |\lambda_{j+1}|$ implies that $\lambda_j = \lambda_{j+1}$. Then the power method applies and finds the eigenvalue $\lambda_1$ and the corresponding eigenvector $\mathbf{v}_1$. Think now what are the eigenvalues and eigenvectors of the matrix $$A - \lambda_1 \mathbf{v}_1\mathbf{v}_1^\top;$$ do you see how to find the second eigenvalue $\lambda_2$ and the corresponding eigenvector?

### **2.2 (0.3 pt)** Find all eigenvalues and eigenvectors of a symmetric matrix with power method
####Explain how to find the second, third etc eigenvalues and the corresponding eigenvectors for a **symmetric** matrix $M$ if the first eigenvalue and the corresponding eigenvector has already been found. Write down the formulas for each step; justify your answer by referring to the corresponding properties of symmetric matrices


---
#### **Your explanations here**
Because  A is symmetric we can apply the Spectral Theorem: $A=Q^TDQ$ for an orthogonal matrix $Q$ and a diagonal matrix $D$ that contains the eigenvalues.
Let $c=Qx(0)$ and we have
$$
\lambda_{k+1} = \frac{c^T D^{2k+1} c}{c^T D^{2k} c }= \frac{\sum_{j=1}^{n} c_{2j} \lambda_{2k+1,j}}{\sum_{j=1}^{n} c_{2j} \lambda_{2k,j}}.
$$

Dividing numerator and denominator by $λ^{2k}_{1}$
  where $λ_1$
  is assumed to be the dominant eigenvalue, we have
$$
\lambda_{k+1} = \lambda_1 * \frac{\sum_{j=2}^{n} c_{j}^2 \left(\frac{\lambda_j}{\lambda_1}\right)^{2k+1}}{\sum_{j=2}^{n} c_{j}^2 \left(\frac{\lambda_j}{\lambda_1}\right)^{2k}} = \lambda_1 * \frac{1+\sum_{j=2}^{n} c_{j}^2 \left(\frac{\lambda_j}{\lambda_1}\right)^{2k+1}}{1 + \sum_{j=2}^{n} c_{j}^2 \left(\frac{\lambda_j}{\lambda_1}\right)^{2k}} = \lambda_1 + \frac{1+O\left(\left|\frac{\lambda_j}{\lambda_1}\right|^{2k+1}\right)}{1 + O\left(\left|\frac{\lambda_j}{\lambda_1}\right|^{2k}\right)}.

$$

From this it follows that $λ^(k)$ would be the approximation of the dominant eigenvalue $λ_1$ of a symmetric matrix using the symmetric Power method, then
$$
λ(k)=λ_1+O\left(\left|\frac{\lambda_2}{\lambda_1}\right|^{2k}\right).
$$
---

###**2.3. (0.3 pts)** Implementation
####Implement the ``symmetric_matrix_find_eig`` function that accepts a **symmetric matrix** $A$ and calculates all eigenpairs of $A$. To test your function, come up with your own $2 \times 2$ symmetric matrix $M_1$ and $3 \times 3$ symmetric matrix $M_2$ for which you can calculate the eigenpairs by hand, and compare the results

In [27]:
def symmetric_power_method(A, n_max: int = 10000):
    x = np.random.rand(A.shape[1])
    x = x / np.linalg.norm(x)
    lambda_k_0 = np.inf
    tolerance = 1e-5

    for _ in range(int(n_max)):
        y = np.dot(A, x)
        lambda_k = np.dot(x.T, y)
        x = y / np.linalg.norm(y)

        if np.abs(lambda_k - lambda_k_0) < tolerance:
            break

    return lambda_k, x

def symmetric_matrix_find_eig(A, n_max: int = 100):
    """
    Return a list of eigenpairs (eigenvalues and eigenvectors)
    of a symmetric matrix A

    Args:
        A - symmetric n x n matrix for which to compute the eigenpairs

    Returns:
        list((eigval, eigvec)) - a list of length n of all eigenpairs stored as
                                 tuples (eigval, eigvec).
    """
    #YOUR CODE STARTS HERE

    a_matrix = A.copy()
    eigen_val_vecs: list[tuple[float, np.array]] = []
    eigen_value_k, v = symmetric_power_method(a_matrix)
    eigen_val_vecs.append((eigen_value_k, v))
    a_matrix -= (eigen_value_k * np.outer(v, v))

    for _ in range(A.shape[1]-1):
        eigen_value_k, v = symmetric_power_method(a_matrix)
        eigen_val_vecs.append((eigen_value_k, v))
        # Use Hotelling's deflation method
        a_matrix -= (eigen_value_k * np.outer(v, v))

    return eigen_val_vecs
    #YOUR CODE ENDS HERE




---
####**Give here the matrices and their eigenpairs calculated by hand**
---

In [28]:
#call the power_method and symmetric_matrix_find_eig functions of symmetric matrices M_1 and M_2 here
M_1 = np.array([[1., 3.],
                [3., 1.]])
M_1_eigpairs = symmetric_matrix_find_eig(M_1)

for e in M_1_eigpairs:
    print(f"eigenvalue: {e[0]}, eigenvector: {e[1]}")
M_2 = np.array([[1., 5., 4.],
                [5., 4., 5.],
                [4., 5., 1.]])
M_2_eigpairs = symmetric_matrix_find_eig(M_2)
print("---")
for e in M_2_eigpairs:
    print(f"eigenvalue: {e[0]}, eigenvector: {e[1]}")

eigenvalue: 4.0, eigenvector: [0.70710678 0.70710678]
eigenvalue: -2.0000000000000004, eigenvector: [-0.70710678  0.70710678]
---
eigenvalue: 11.588723439378912, eigenvector: [0.5173332  0.68171308 0.5173332 ]
eigenvalue: -3.0, eigenvector: [-7.07106781e-01 -1.25965539e-16  7.07106781e-01]
eigenvalue: -2.5887234393789122, eigenvector: [-0.48204394  0.73161963 -0.48204394]


#### Test your implementation:

In [29]:
def test_symmetric_matrix_find_eig(A, eigenpairs):

    eigenvals_found = np.array([e[0] for e in eigenpairs])
    eigenvals_ref, _ = np.linalg.eigh(A)
    assert np.allclose(np.sort(eigenvals_found), np.sort(eigenvals_ref)),\
    f"Incorrect eigenvalue found: {eigenvals_found} differs from {eigenvals_ref}"
    print("test_power_method passed successfully")

In [30]:
A_test_3x3 = (10*np.random.rand(3,3) - 5)
A_test_5x5 = (10*np.random.rand(5,5) - 5)
A_test_10x10 = (10*np.random.rand(10,10) - 5)
A_sym_test_3x3 = A_test_3x3 + A_test_3x3.T
A_sym_test_5x5 = A_test_5x5 + A_test_5x5.T
A_sym_test_10x10 = A_test_10x10 + A_test_10x10.T

test_symmetric_matrix_find_eig(A_sym_test_3x3, symmetric_matrix_find_eig(A_sym_test_3x3))
test_symmetric_matrix_find_eig(A_sym_test_5x5, symmetric_matrix_find_eig(A_sym_test_5x5))
test_symmetric_matrix_find_eig(A_sym_test_10x10, symmetric_matrix_find_eig(A_sym_test_10x10))

test_power_method passed successfully
test_power_method passed successfully
test_power_method passed successfully


###**2.4 (0.4 pt)** Why is symmetry important?
####Explain why this method will not work for **non-symmetric** matrices $A$. Find a $3\times3$ example of diagonalizable matrix $A$ for which ``symmetric_matrix_find_eig`` function fails to find its correct eigenvalues and eigenvectors

The symmetry is important, because for non-symmetric matrix the symmetric power method simply would not work, due to the fact that we cannot apply the spectral theorem as a first step of given algorithm.
The symmetric power method (not deflation method) has faster convergence (lower complexity)

For example, given non symmetric matrix A:
[[-1., 3., -1.],
 [-3., 5., -1.],
 [-3., 3., 1.]]
With eigenvalues 1, 2 and 2, which is diagonazible, we would get wrong eigenvalues from our symmetric matrix.
The method fails not because the matrix A is diagonalizible, but because it's not symmetric and deflation algorithm cannot guarantee that the following $B$ matrix: $B = A - \lambda * v * v^T $ has eigenvalues smaller than the already found one, resulting in not correct eigenvalues and eigenvectors, because it cannot converge. (You can check that the cell below this one, has called symmetric power method, which does not use deflation, and it has found correct eigenvalue (2))
Henceforth, in order to fix this issue we would have to either use another kind of method than power method, its modification, OR use another deflation technic for non-symmetric matrices


In [31]:
#YOUR CODE STARTS HERE
A_non_symmetric = np.array([[-1., 3., -1.],
                            [-3., 5., -1.],
                            [-3., 3., 1.]])
test_symmetric_matrix_find_eig(A_non_symmetric, symmetric_matrix_find_eig(A_non_symmetric))
#YOUR CODE ENDS HERE

AssertionError: Incorrect eigenvalue found: [2. 2. 1.] differs from [-3.24610782 -0.14681945  8.39292727]

In [32]:
symmetric_power_method(A_non_symmetric)

(2.0, array([ 0.7217173 ,  0.681695  , -0.12006691]))

## 3. Inverse power method (0.6 pt)

###3.1 The main idea
#### Now you will try to find non-dominant eigenpairs for a generic matrix $A$ using the **inverse power method / inverse iteration method**. This method finds an eigenvalue of $A$ that is the closest one to a given guess value $\mu$, along with the corresponding eigenvector. By trying different $\mu$, we will find all simple eigenvalue/eigenvector pairs of $A$, not just the dominant one.

####The idea is that if $\lambda_*$ is the eigenvalue of $A$ that is the closest one to $\mu$, then $(\lambda - \mu)^{-1}$ is the dominant eigenvalue of the matrix $B:=(A - \mu I)^{-1}$, while the corresponding eigenvector $\mathbf{v}_*$ of $B$ is also an eigenvector of $A$ corresponding to $\lambda_*$.

####The natural approach is to find first the dominant eigenvalue $\lambda_1$ of $A$; then all the remaining eigenvalues satisfy  $$\forall j\ne1: |\lambda_j| \lt |\lambda_1|, $$ and we can apply a random search for $|\mu| < |\lambda_1|$ and call the **inverse power method** for each such $\mu$. This way, we will identify all the **simple** eigenvalues and the corresponding eigenvectors of $A$

>**Note:** Here, it may be necessary to work with complex numbers. In that case, the corresponding changes to the power method must be made (recall how the scalar product in $\mathbb{C}^k$ differs from that in $\mathbb{R}^k$)

###**3.2 (0.4 pt)** Implement the inverse power method
#### Function ``inverse_power_method``:

In [33]:
from scipy.linalg import solve


def inverse_power_method(A, approx_eigenvalue, start_eigvector=None, max_iter=100):
    """
    Return the largest eigenvalue and it's corresponding eigenvector
    using power method.

    Args:
        A - matrix for which to compute the eigenpair
        approx_eigenvalue - the \mu parameter, a value closest to some eigenvalue l,
                            which will be returned together with it's eigenvector
        start_eigvector(optional) - eigenvector used for initialization
                                    on the first step of power iteration algorithm.
                                    Can be an approximation of the real eigenvector,
                                    but doesn't have to be. Defaults to None.
                                    In such case, the initial vector will be randomized.
        max_iter(optional) - maximum number of iterations for the power method.
    Returns:
        (eigval, eigvec) - a pair of an eigenvalue closest to approx_eigenvalue
                           and it's eigenvector.
    """

    #YOUR CODE STARTS HERE
    n, m = A.shape

    # Set initial vector
    if start_eigvector is None:
        start_eigvector = np.random.rand(n, 1)
    else:
        start_eigvector = np.array(start_eigvector).reshape(n, 1)

    # Inverse power iteration
    for _ in range(int(max_iter)):
        # Solve the linear system (A - approx_eigenvalue * I) * x = start_eigvector
        lambda_reg = 1e-3  # Small regularization parameter
        next_eigvector = np.linalg.solve(A - approx_eigenvalue * np.identity(n) + lambda_reg * np.identity(n), start_eigvector)

        # Calculate the eigenvalue estimate using the Rayleigh quotient
        eigenvalue = np.dot(next_eigvector.T, np.dot(A, next_eigvector)) / np.dot(next_eigvector.T, next_eigvector)

        # Normalize the vector
        next_eigvector = next_eigvector / np.linalg.norm(next_eigvector)

        # Check for convergence
        if np.linalg.norm(next_eigvector - start_eigvector) < 1e-6:
            break

        start_eigvector = next_eigvector

    return eigenvalue[0, 0], start_eigvector.flatten()
    #YOUR CODE ENDS HERE

###**3.3 (0.2pt):** Testing the ``inverse_power_method``
#### Apply the method to find a few (at least 3) eigenvalues and eigenvectors of the matrix $A$ defined as $$ A = P D P^{-1},$$ where $$ D = \mbox{diag}(0,1,2,3,\cdots,9)$$ is diagonal and $P$ is a random $10 \times 10$ matrix. Explain why the diagonal entries of $D$ are eigenvalues of $A$ and why the columns of $P$ are the corresponding eigenvectors. Use that observation to test the found eigenvalues and eigenvectors of $A$

For diagonal entries D:
For a diagonal matrix $D$ = $diag(\lambda_1, \lambda_2, ..., \lambda_n)$ the eigenvalues are just the elements of a basis
When a matrix is multiplied by a basis vector, the result is scaled corresponding to the diagonal entries.
When A operates on the columns of P, the effect is the same as of D on the basis, henceforth the D are the eigenvalues of A.
For P as corresponding Eigenvectors:
If a matrix $\(A\)$ can be diagonalized as $\(A = P D P^{-1}\)$, then the columns of $\(P\)$ are the eigenvectors of $\(A\)$, and $\(D\)$ contains the corresponding eigenvalues.
The columns of $\(P\)$ form a basis for the eigenspace corresponding to the eigenvalues on the diagonal of $\(D\)$.
Each column of $\(P\)$ corresponds to an eigenvector, and the linear combination of these columns can represent any vector in the eigenspace.
When $\(A\)$ operates on the columns of $\(P\)$, each column remains an eigenvector of $\(A\)$, and the corresponding eigenvalue is obtained from the diagonal entry of $\(D\)$.

The diagonal entries of $\(D\)$ are eigenvalues of $\(A\)$ since they characterize the scaling effect on the standard basis vectors when $\(A\)$ is applied. The columns of $\(P\)$ are the corresponding eigenvectors because they form a basis for the eigenspace, and when $\(A\)$ operates on these columns, the result is a scaled version of the corresponding column.

In [35]:
D = np.diag([0,1,2,3,4,5,6,7,8,9])
P = (20*np.random.rand(10,10) - 5)
#define A through known eigenvalues and random eigenvectors
A = P @ D @ np.linalg.inv(P)
#YOUR CODE STARTS HERE
for i in range(A.shape[1]):
    eigenvalue, eigenvector = inverse_power_method(A, D[i, i])
    print(f"""
I: {i + 1}
Eigenvalue
Actual: {D[i, i]}
Computed: {eigenvalue}
Eigenvector
Actual: {P[:, i]/np.linalg.norm(P[:, i])}
Computed: {eigenvector}
    """)
#YOUR CODE ENDS HERE


I: 1
Eigenvalue
Actual: 0
Computed: 5.170427635287684e-10
Eigenvector
Actual: [ 0.46547134  0.56005314  0.17713856  0.27894406  0.25140177 -0.10312496
  0.28509268  0.13328012  0.30344001  0.30908604]
Computed: [ 0.46547163  0.56005283  0.17713869  0.27894385  0.25140165 -0.10312463
  0.28509295  0.13328049  0.30343981  0.30908627]
    

I: 2
Eigenvalue
Actual: 1
Computed: 1.000000000046993
Eigenvector
Actual: [ 0.5803764   0.00118026  0.22930106 -0.04635418 -0.01567917  0.22514519
  0.45012993  0.4291253  -0.0469584   0.41052088]
Computed: [ 0.58037632  0.00118003  0.229301   -0.04635412 -0.01567932  0.2251452
  0.45013007  0.42912518 -0.04695861  0.41052096]
    

I: 3
Eigenvalue
Actual: 2
Computed: 1.999999999998767
Eigenvector
Actual: [ 0.50459035  0.30325805  0.03291481  0.33554827 -0.11199695 -0.16102761
  0.45650053  0.01601152 -0.09819813  0.53196044]
Computed: [-0.50459035 -0.30325805 -0.03291481 -0.33554828  0.11199695  0.16102761
 -0.45650054 -0.01601152  0.09819813 -0.5319

##4. Conclusions (0.4 pts)

#### Summarize in a few sentences what you learned by completing this task. Mention the difficulties you might have faced with, any properties/facts that you now understand better (if any)

---
In this task I've learned how to calculate the eigenvalues and eigenvectors, the role and importance of dominant eigenvalue. For instance, I familiarized myself with deflation technique, to calculate further eigenvalues smaller than the dominate one, using the same power method. Furthermore, I understood and implemented symmetric power method which is more efficient
for symmetric matrices, but not as general as the simple power method.
Also, I checked cases when these methods do not work properly, and when they might not converge, for instance classical power methods works bad with initial estimate vector being
orthogonal to true eigenvector corresponding dominating eigenvalue.
---